In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path
import glob
import pickle
import time
import multiprocessing

### Aggregate bootleg scores

In [ ]:
def loadPickle(pkl_file):
    with open(pkl_file, 'rb') as f:
        d = pickle.load(f)
    return d

In [ ]:
def savePickle(pkl_file, d):
    with open(pkl_file, 'wb') as f:
        pickle.dump(d, f)

In [ ]:
def aggregateBootlegScores(score_feat_dir, outdir):
    if not os.path.isdir(outdir):
        os.makedirs(outdir)
    for curdir in glob.glob('{}/*'.format(score_feat_dir)):
        
        # concatenate bootleg score fragments
        pieceid = os.path.basename(curdir)
        outfile = '{}/{}.pkl'.format(outdir, pieceid)
        if os.path.exists(outfile):
            continue
        numFiles = len(glob.glob('{}/*.pkl'.format(curdir)))
        fragments = []
        for i in range(numFiles):
            pkl_file = '{}/{}-{}.pkl'.format(curdir, pieceid, i)
            d = loadPickle(pkl_file)
            assert 'bscore' in d, "bscore key not found in {}".format(pkl_file)
            bscore = d['bscore']
            if bscore is not None and isinstance(bscore, np.ndarray) and bscore.ndim == 2 and bscore.shape[0] == 62:
                fragments.append(bscore)
        
        # save global bootleg score
        if len(fragments) == 0:
            savePickle(outfile, None)
        else:
            concat = np.hstack(fragments)
            savePickle(outfile, concat)

In [ ]:
score_feat_dir = 'score_feat' # directory containing sheet music bootleg scores
score_feat_agg_dir = 'score_feat_agg' # directory containing aggregated bootleg scores
aggregateBootlegScores(score_feat_dir, score_feat_agg_dir)

### Construct Reverse Index

In [ ]:
def constructSingleIndex(score_feat_dir, savefile = None):
    num_files_p = len(glob.glob('{}/p*.pkl'.format(score_feat_dir)))
    num_files_f = len(glob.glob('{}/f*.pkl'.format(score_feat_dir)))
    num_files_total = num_files_p + num_files_f
    d = {} # key: 64-bit fp value, value: list of (pieceNum, offset) tuples
    for pieceNum in range(1, num_files_total + 1):
        if pieceNum < num_files_p + 1:
            curfile = '{}/p{}.pkl'.format(score_feat_dir, pieceNum)
        else:
            curfile = '{}/f{}.pkl'.format(score_feat_dir, pieceNum - num_files_p)
        if not os.path.exists(curfile):
            assert False, "Cannot find {}".format(curfile)
        fps = extractFingerprints_sheet(curfile)
        if len(fps) == 0:
            continue
        for i, fp in enumerate(fps):
            if fp not in d:
                d[fp] = []
            if i < len(fps) - 2:
                d[fp].append((pieceNum, i, fps[i+1], fps[i+2]))
            else:
                d[fp].append((pieceNum, i, 0, 0))

    if savefile:
        with open(savefile, 'wb') as f:
            pickle.dump([d, num_files_total], f)
    return d, num_files_total

In [ ]:
def extractFingerprints_sheet(pkl_file):
    '''
    Convert binary matrix to list of 64-bit int fingerprints.
    '''
    D = loadPickle(pkl_file) # 62 x N binary matrix
    if D is None:
        return []
    numBits = D.shape[0]
    if D.shape[1] <= 1: # empty or contains single filler column
        return []
    assert numBits < 64, "Number of bits must be less than 64."
    mirrored = mirrorBothHands(D)
    mask = np.power(2, np.arange(numBits)).reshape((1,-1))
    fps = np.squeeze(mask @ mirrored)
    return fps

In [ ]:
def mirrorBothHands(bscore):
    mirrored = np.maximum(bscore[18:28,:], bscore[28:38,:]) # overlap between hands is E3 to G4 inclusive
    bscore[18:28,:] = mirrored
    bscore[28:38,:] = mirrored
    return bscore

In [ ]:
def loadSingleIndexDB(pkl_file):
    d = loadPickle(pkl_file)
    dsingle = d[0]
    numFiles = d[1]
    return dsingle, numFiles

In [ ]:
singleIndexFile = 'dbSingle.pkl'
constructSingleIndex(score_feat_agg_dir, singleIndexFile)
dsingle, dbSize = loadSingleIndexDB(singleIndexFile)

In [ ]:
def constructTripleIndex(d_singles, thresh):
    d_triples = {}
    for fp in d_singles:
        listTups = d_singles[fp]
        if len(listTups) >= thresh:
            for (pieceNum, offset, fpA, fpB) in listTups:
                fpNew = fp + fpA // 2 + fpB // 4
                if fpNew not in d_triples:
                    d_triples[fpNew] = []
                d_triples[fpNew].append((pieceNum, offset))
    return d_triples

In [ ]:
def saveFullIndexDB(pkl_file, dbsingle, dbtriple, dbsize, dbthresh):
    with open(pkl_file, 'wb') as f:
        pickle.dump({'dbsingle': dbsingle, 'dbtriple': dbtriple, 'dbsize': dbsize, 'dbthresh': dbthresh}, f)

In [ ]:
def loadFullIndexDB(pkl_file):
    with open(pkl_file, 'rb') as f:
        d = pickle.load(f)
    dbsingle = d['dbsingle']
    dbtriple = d['dbtriple']
    dbsize = d['dbsize']
    dbthresh = d['dbthresh']
    return (dbsingle, dbtriple, dbsize, dbthresh)

In [ ]:
dbThresh = 8000
dtriple = constructTripleIndex(dsingle, dbThresh)

In [ ]:
fullIndexFile = 'dbAll.pkl'
saveFullIndexDB(fullIndexFile, dsingle, dtriple, dbSize, dbThresh)
dbInfo = loadFullIndexDB(fullIndexFile)

In [ ]:
def getFingerprintStats(d):
    
    # compute frequency of fingerprint occurrences
    freqs = []
    for key in d:
        freqs.append(len(d[key]))
    freqs = np.array(freqs)
    freqs_sorted = sorted(freqs)
    
    # print useful stats
    print('Number of unique fingerprints: {}'.format(len(freqs)))
    print('Number of fingerprints that occur exactly once: {}'.format(np.sum(freqs == 1)))
    print('5 most frequently occuring fingerprints occur _ times: {}'.format(freqs_sorted[::-1][0:5]))

In [ ]:
getFingerprintStats(dbInfo[0]) # single fingerprints

In [ ]:
getFingerprintStats(dbInfo[1]) # triple fingerprints

### Process Query

In [ ]:
def processMidiQuery(midi_bscore_file, dbInfo, sampleDur, nsamples, binSize, savefile = None):
    
    # load DB, start profiling
    #print('Processing {}'.format(midi_bscore_file))
    profileStart = time.time()
    
    # generate random samples from sharp & flat bscores
    d = loadPickle(midi_bscore_file)
    bscoreSharp = d['bscoreSharp']
    bscoreFlat = d['bscoreFlat']
    bscoreLength = bscoreSharp.shape[1]
    sorted_sharp = []
    sorted_flat = []
    scores_sharp = []
    scores_flat = []
    np.random.seed(0)
    for i in range(nsamples):
        if sampleDur == -1: # use entire midi bscore
            bscoreSharp_sample = bscoreSharp
            bscoreFlat_sample = bscoreFlat
        else: # use randomly selected sample
            offset = np.random.randint(max(bscoreLength - sampleDur + 1, 1))
            bscoreSharp_sample = bscoreSharp[:,offset:offset+sampleDur]
            bscoreFlat_sample = bscoreFlat[:,offset:offset+sampleDur]    
        sorted_pids_sharp, scores_pids_sharp = processMidiBscore(bscoreSharp_sample, dbInfo, binSize)
        sorted_pids_flat, scores_pids_flat = processMidiBscore(bscoreFlat_sample, dbInfo, binSize)
        sorted_sharp.append(sorted_pids_sharp)
        scores_sharp.append(scores_pids_sharp)
        sorted_flat.append(sorted_pids_flat)
        scores_flat.append(scores_pids_flat)
        
    # select the one with the higher score
    sorted_both = []
    for i in range(nsamples):
        score_sharp = np.max(scores_sharp[i])
        score_flat  = np.max(scores_flat[i])
        if score_sharp > score_flat:
            sorted_both.append(sorted_sharp[i])
        else:
            sorted_both.append(sorted_flat[i])
            
    profileEnd = time.time()
    profileDur = profileEnd - profileStart
    
    # save results
    sorted_both = np.array(sorted_both)
    sorted_sharp = np.array(sorted_sharp)
    sorted_flat = np.array(sorted_flat)
    scores_sharp = np.array(scores_sharp)
    scores_flat = np.array(scores_flat)
    results = {}
    results['sorted_both'] = sorted_both
    results['sorted_sharp'] = sorted_sharp
    results['sorted_flat'] = sorted_flat
    results['scores_sharp'] = scores_sharp
    results['scores_flat'] = scores_flat
    results['query'] = midi_bscore_file
    results['dbSize'] = dbInfo[2]
    results['dbThresh'] = dbInfo[3]
    results['sampleDur'] = sampleDur
    results['nsamples'] = nsamples
    results['binSize'] = binSize
    results['profileDur'] = profileDur
    
    if savefile:
        with open(savefile, 'wb') as f:
            pickle.dump(results, f)
    
    return sorted_both, sorted_sharp, sorted_flat, scores_sharp, scores_flat

In [ ]:
def processMidiBscore(bscore, dbInfo, binSize):
    
    (dbSingle, dbTriple, dbSize, dbThresh) = dbInfo
    fps = extractFingerprints_midi(bscore)
    
    # construct histogram of offsets
    h = {}
    for i, fp in enumerate(fps):
        if fp in dbSingle:
            if len(dbSingle[fp]) < dbThresh: # rare fp value, use single fp matches
                matches = dbSingle[fp] # list of (pieceNum, offset, fpNext, fpNextNext)
            elif i < len(fps) - 2: # common fp value, use triple fp matches
                fpTriple = fp + fps[i+1] // 2 + fps[i+2] // 4
                if fpTriple in dbTriple:
                    matches = dbTriple[fpTriple] # list of (pieceNum, offset)
                else:
                    continue
            else: # common fp value but at end of sequence, ignore
                continue
            for m in matches:
                pieceNum = m[0]
                refOffset = m[1]
                offsetDiff = (refOffset - i) // binSize
                if pieceNum not in h:
                    h[pieceNum] = {}
                if offsetDiff not in h[pieceNum]:
                    h[pieceNum][offsetDiff] = 0
                h[pieceNum][offsetDiff] += 1
    
    # sort db items by match score
    scores = [0] * (dbSize+1) # numbering starts from 1, ignore scores[0]
    for pieceNum in range(1, dbSize + 1):
        if pieceNum in h:
            scores[pieceNum] = np.max([h[pieceNum][offset] for offset in h[pieceNum]])
    
    resultList = np.argsort(scores)[::-1] # sorted from highest score to lowest score
    
    return resultList, scores

In [ ]:
def extractFingerprints_midi(D):
    '''
    Extract list of fingerprint tuples given a MIDI bootleg score matrix.
    '''
    numBits = D.shape[0]
    if D.shape[1] <= 1: # empty or contains single filler column
        return []
    assert numBits < 64, "Number of bits must be less than 64."
    mask = np.power(2, np.arange(numBits)).reshape((1,-1))
    fps = np.squeeze(mask @ D)
    
    return fps

In [ ]:
midi_bscore_file = 'midi_feat/p1.pkl'
sampleDur = 20
nsamples = 2
binSize = 10

In [ ]:
sorted_both, sorted_sharp, sorted_flat, scores_sharp, scores_flat = processMidiQuery(midi_bscore_file, dbInfo, sampleDur, nsamples, binSize)

### Process All Queries

In [ ]:
def processAllQueries(midi_bscore_dir, dbInfo, sampleDur, nsamples, binSize, outdir):
    if not os.path.isdir(outdir):
        os.makedirs(outdir)

    numQueryFiles = len(glob.glob('{}/*.pkl'.format(midi_bscore_dir)))
    for i in range(1, numQueryFiles+1):
        midi_bscore_file = '{}/p{}.pkl'.format(midi_bscore_dir, i)
        hyp_file = '{}/p{}.hyp'.format(outdir, i)
        if os.path.exists(hyp_file):
            print('Skipping {}'.format(midi_bscore_file))
        else:
            print('Processing {}'.format(midi_bscore_file))
            processMidiQuery(midi_bscore_file, dbInfo, sampleDur, nsamples, binSize, hyp_file)

In [ ]:
midi_bscore_dir = 'midi_feat'
sampleDur = 100
nsamples = 10
binSize = 10
results_dir = 'hyps/sample{}'.format(sampleDur)
#processAllQueries(midi_bscore_dir, dbInfo, sampleDur, nsamples, binSize, results_dir)

In [ ]:
processAllQueries(midi_bscore_dir, dbInfo, 10, nsamples, binSize, 'hyps/sample10')
processAllQueries(midi_bscore_dir, dbInfo, 20, nsamples, binSize, 'hyps/sample20')
processAllQueries(midi_bscore_dir, dbInfo, 50, nsamples, binSize, 'hyps/sample50')
processAllQueries(midi_bscore_dir, dbInfo, 100, nsamples, binSize, 'hyps/sample100')
processAllQueries(midi_bscore_dir, dbInfo, 200, nsamples, binSize, 'hyps/sample200')
processAllQueries(midi_bscore_dir, dbInfo, 500, nsamples, binSize, 'hyps/sample500')
processAllQueries(midi_bscore_dir, dbInfo, 1000, nsamples, binSize, 'hyps/sample1000')
processAllQueries(midi_bscore_dir, dbInfo, 100000, nsamples, binSize, 'hyps/sample100000')

In [ ]:
# # process all queries
# midi_bscore_dir = 'midi_feat'
# #dbfile = 'dbAll.pkl'
# sampleDur = 100
# nsamples = 10
# binSize = 10
# results_dir = 'hyps/sample{}'.format(sampleDur)

# # prep output directory
# if not os.path.isdir(results_dir):
#     os.makedirs(results_dir)

# # number of cores to use
# n_cores = 4 #multiprocessing.cpu_count()

# # prep inputs for parallelization
# inputs = []
# numQueryFiles = len(glob.glob('{}/*.pkl'.format(midi_bscore_dir)))
# for i in range(1, numQueryFiles+1):
#     midi_bscore_file = '{}/p{}.pkl'.format(midi_bscore_dir, i)
#     hyp_file = '{}/p{}.hyp'.format(results_dir, i)
#     if os.path.exists(hyp_file):
#         print('Skipping {}'.format(midi_bscore_file))
#     else:
#         inputs.append((midi_bscore_file, dbInfo, sampleDur, nsamples, binSize, hyp_file))

# # process queries in parallel
# pool = multiprocessing.Pool(processes=n_cores)
# outputs = list(pool.starmap(processMidiQuery, inputs))